In [ ]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import os
from tqdm import tqdm
import re

import pandas as pd
# pandas version: 0.25.1

import numpy as np
# numpy version: 1.16.1

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.models import Sequential
from keras.layers.core import Dense
# keras version: 2.2.4
# Using TensorFlow backend in Keras (1.13.1)

import innvestigate
# innvestigate version: 1.0.8

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import ngrams, FreqDist
# nltk version: 3.4.5

from lime.lime_text import LimeTextExplainer
# lime version: 0.2.0.1

from math import ceil

<h1>Model</h1>

In [ ]:
# Data: https://ai.stanford.edu/~amaas/data/sentiment/
base_folder = '../aclImdb/'

def parse_folder(name):
    data = []
    for verdict in ('neg', 'pos'):
        for file in tqdm(glob(os.path.join(name, verdict, '*.txt'))):
            data.append({
                'text': open(file, encoding='utf8').read(),
                'verdict': verdict
            })
    return pd.DataFrame(data)

df_train = parse_folder(base_folder+'train/')
df_test = parse_folder(base_folder+'test/')

df = pd.concat([df_train, df_test])
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)
df = df.sample(frac=1, random_state=7)

In [ ]:
df.head()

In [ ]:
sentences = list(df['text'])
sentiment = list(df['verdict'])

In [ ]:
sentences_train, sentences_test, sentiment_train, sentiment_test = train_test_split(sentences, sentiment, test_size=0.20, random_state=7)

In [ ]:
# pos:1, neg:0
y_train = []
for x in sentiment_train:
    if x == 'pos':
        y_train.append(1)
    else:
        y_train.append(0)
y_train = np.array(y_train)

y_test = []
for x in sentiment_test:
    if x == 'pos':
        y_test.append(1)
    else:
        y_test.append(0)
y_test = np.array(y_test)

In [ ]:
stop_words = set(stopwords.words("english"))
ps = PorterStemmer()

def preprocess_text(sen):
    sentence = re.sub('[^A-Za-z0-9]', ' ', sen)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    sentence_clean = ""
    
    for w in word_tokenize(sentence):
        if (w not in stop_words) & (len(w)>2):
            sentence_clean = sentence_clean + " " + ps.stem(w)
    return sentence_clean.strip()

In [ ]:
X_preprocess_train = []
for sen in tqdm(sentences_train):
    X_preprocess_train.append(preprocess_text(sen))
    
X_preprocess_test = []
for sen in tqdm(sentences_test):
    X_preprocess_test.append(preprocess_text(sen))

In [ ]:
vectorizer = TfidfVectorizer(max_features=500)
X_vectorized_train = vectorizer.fit_transform(X_preprocess_train)
X_vectorized_test = vectorizer.transform(X_preprocess_test)

In [ ]:
vocab_words = list(vectorizer.vocabulary_.keys())
vocab_idx = list(vectorizer.vocabulary_.values())
vocab_inv = {}
for i, idx in enumerate(vocab_idx):
    vocab_inv[idx] = vocab_words[i]

In [ ]:
model = Sequential([
  Dense(100, activation='relu', input_shape=(500,)),
  Dense(30, activation='relu'),
  Dense(1, activation='sigmoid'),
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_vectorized_train, y_train , epochs=3, batch_size=32)

In [ ]:
model.evaluate(X_vectorized_train, y_train)

In [ ]:
model.evaluate(X_vectorized_test, y_test)

<h1>Initilizing XAI Techniques</h1>

In [ ]:
def predict_proba_fn(text_list):
    
    X_preprocess = []
    for sen in tqdm(text_list):
        X_preprocess.append(preprocess_text(sen))
    
    X_vectorized = vectorizer.transform(X_preprocess)
    proba = model.predict_proba(X_vectorized)
    proba_not = 1-proba
    proba = np.concatenate((proba, proba_not), axis=1)

    return proba

In [ ]:
explainer = LimeTextExplainer(class_names=['pos', 'neg'])

In [ ]:
innvestigate_method = 'lrp.z'
analyzer = innvestigate.create_analyzer(innvestigate_method, model)

<h1>Case Selection</h1>

In [ ]:
np.random.seed(7)
random_cases = list(np.random.random_integers(0,9999,size=[500,]))

In [ ]:
experimental_data_idx = list(range(0,len(random_cases)))

In [ ]:
case_no = random_cases[0]

In [ ]:
sentences_test[case_no]

In [ ]:
experimental_data_text = []
for case_no_ in random_cases:
    experimental_data_text.append(sentences_test[case_no_])

In [ ]:
print('True:', sentiment_test[case_no])
print('Predicted:', model.predict_classes(X_vectorized_test[case_no]))

In [ ]:
model.predict_classes(X_vectorized_test[case_no])[0][0]

In [ ]:
experimental_data_y = []
for case_no_ in random_cases:
    experimental_data_y.append(sentiment_test[case_no_])

In [ ]:
experimental_data_y_hat = []
for case_no_ in random_cases:
    p = model.predict_classes(X_vectorized_test[case_no_])[0][0]
    if p == 1:
        experimental_data_y_hat.append('pos')
    else:
        experimental_data_y_hat.append('neg')

<h1>LIME</h1>

In [ ]:
exp = explainer.explain_instance(sentences_test[case_no], classifier_fn=predict_proba_fn,num_features=10)
a = exp.as_list()

pos_score = []
pos_words = []
neg_score = []
neg_words = []
for j in a:
    if j[1] <0:
        pos_score.append(round(j[1]*-1,5))
        pos_words.append(j[0])
    else:
        neg_score.append(round(j[1]*-1,5))
        neg_words.append(j[0])

print("\nPOS WORD CONTRIBUTE SCORE:-")
for i, score in enumerate(pos_score):
    print(pos_words[i],':', score)
    
print("\nNEG WORD CONTRIBUTE SCORE:-")
for i, score in enumerate(neg_score):
    print(neg_words[i],':', score)

In [ ]:
import random
from IPython.core.display import display, HTML

In [ ]:
def lime_highlighter_pos(p, ma, mi):
    w = p[0]
    s = p[1]
    cl = ma-mi
    g_ = (s-mi)/cl
    c_ = ((1-g_)*50)+45
    color =  "hsl(133, 100%, "+str(c_)+"%)"
    word = '<span style="background-color:' +color+ '">' +w+ '</span>'
    return word

text_pos = ' '.join([lime_highlighter_pos(p, max(pos_score),min(pos_score)) for p in zip(pos_words, pos_score)])
display(HTML(text_pos))

In [ ]:
def lime_highlighter_neg(p, ma, mi):
    w = p[0]
    s = p[1]
    cl = ma-mi
    g_ = (s-mi)/cl
    c_ = ((1-g_)*50)+45
    color =  "hsl(0, 100%, "+str(c_)+"%)"
    word = '<span style="background-color:' +color+ '">' +w+ '</span>'
    return word

text_neg = ' '.join([lime_highlighter_neg(p, max(list(np.abs(neg_score))),min(list(np.abs(neg_score)))) for p in zip(neg_words, list(np.abs(neg_score)))])
display(HTML(text_neg))

In [ ]:
def e_lime(case_no):
    exp = explainer.explain_instance(sentences_test[case_no], classifier_fn=predict_proba_fn,num_features=10)
    a = exp.as_list()

    pos_score = []
    pos_words = []
    neg_score = []
    neg_words = []
    for j in a:
        if j[1] <0:
            pos_score.append(round(j[1]*-1,5))
            pos_words.append(j[0])
        else:
            neg_score.append(round(j[1]*-1,5))
            neg_words.append(j[0])

    print("POS WORD CONTRIBUTE:")
    text_pos = ' '.join([lime_highlighter_pos(p, max(pos_score),min(pos_score)) for p in zip(pos_words, pos_score)])
    display(HTML(text_pos))
    
    print("NEG WORD CONTRIBUTE:")
    text_neg = ' '.join([lime_highlighter_neg(p, max(list(np.abs(neg_score))),min(list(np.abs(neg_score)))) for p in zip(neg_words, list(np.abs(neg_score)))])
    display(HTML(text_neg))

In [ ]:
cn = -1
experimental_data_e_lime = []
for case_no_ in tqdm(random_cases):
    cn =cn+1
    if (cn>=300) & (cn<=319):
        experimental_data_e_lime.append(e_lime(case_no_))
        print('id:', cn)

<h1>LRP</h1>

In [ ]:
case = X_vectorized_test[case_no]
scores = np.squeeze(analyzer.analyze(case.toarray()))

print("\nWORD CONTRIBUTE SCORE:-")
top_set = []
scores = np.abs(scores)
ids = np.flip(scores.argsort()[-10:])
words_list = []
scores_list = []
for i in ids:
    print(vocab_inv[i],':', round(scores[i],5))
    words_list.append(vocab_inv[i])
    scores_list.append(round(scores[i],5))

In [ ]:
def lrp_highlighter(p, ma, mi):
    w = p[0]
    s = p[1]
    cl = ma-mi
    g_ = (s-mi)/cl
    c_ = ((1-g_)*50)+45
    color =  "hsl(202, 100%, "+str(c_)+"%)"
    word = '<span style="background-color:' +color+ '">' +w+ '</span>'
    return word

text_high = ' '.join([lrp_highlighter(p, max(scores_list),min(scores_list)) for p in zip(words_list, scores_list)])
display(HTML(text_high))

In [ ]:
def e_lrp(case_no):
    case = X_vectorized_test[case_no]
    scores = np.squeeze(analyzer.analyze(case.toarray()))
    e_lrp_str = []
    scores = np.abs(scores)
    ids = np.flip(scores.argsort()[-10:])
    
    words_list = []
    scores_list = []
    for i in ids:
        e_lrp_str.append((vocab_inv[i], round(scores[i],5)))
        words_list.append(vocab_inv[i])
        scores_list.append(round(scores[i],5))
        
    print("WORD CONTRIBUTE:")
    text_high = ' '.join([lrp_highlighter(p, max(scores_list),min(scores_list)) for p in zip(words_list, scores_list)])
    display(HTML(text_high))

In [ ]:
cn = -1
experimental_data_e_lrp = []
for case_no_ in tqdm(random_cases):
    cn =cn+1
    if (cn>=300) & (cn<=319):
        print('id:', cn)
        experimental_data_e_lrp.append(e_lrp(case_no_))

<h1>CounterFactual example</h1>

In [ ]:
sentences_case = sentences_test[case_no]
sentences_case_word_tokens = word_tokenize(sentences_case)
sentences_case_length = len(sentences_case_word_tokens)
new_words_count = ceil(sentences_case_length/10)

random_words_index = list(np.random.random_integers(0,len(vocab_words)-1,size=[new_words_count,]))
new_words = []
for i in random_words_index:
    new_words.append(vocab_words[i])
    
random_location_index = list(np.random.random_integers(0,sentences_case_length,size=[new_words_count,]))

sentences_new = ""
for i, w in enumerate(sentences_case_word_tokens):
    if i in random_location_index:
        sentences_new = sentences_new + " **" + new_words[np.where(np.array(random_location_index)==i)[0][0]] + "**"
    else:
        sentences_new = sentences_new + " " + w
sentences_new = sentences_new.strip()

print(sentences_new)
text_list = [sentences_new]
X_preprocess = []
for sen in tqdm(text_list):
    X_preprocess.append(preprocess_text(sen))

X_vectorized = vectorizer.transform(X_preprocess)

clas = model.predict_classes(X_vectorized)
print('\nModel Prediction:', clas)

In [ ]:
def text_purt_clas(case_no):
    sentences_case = sentences_test[case_no]
    sentences_case_word_tokens = word_tokenize(sentences_case)
    sentences_case_length = len(sentences_case_word_tokens)
    new_words_count = ceil(sentences_case_length/10)

    random_words_index = list(np.random.random_integers(0,len(vocab_words)-1,size=[new_words_count,]))
    new_words = []
    for i in random_words_index:
        new_words.append(vocab_words[i])

    random_location_index = list(np.random.random_integers(0,sentences_case_length,size=[new_words_count,]))

    sentences_new = ""
    for i, w in enumerate(sentences_case_word_tokens):
        if i in random_location_index:
            sentences_new = sentences_new + " **" + new_words[np.where(np.array(random_location_index)==i)[0][0]] + "**"
        else:
            sentences_new = sentences_new + " " + w
    sentences_new = sentences_new.strip()

    text_list = [sentences_new]
    X_preprocess = []
    for sen in tqdm(text_list):
        X_preprocess.append(preprocess_text(sen))

    X_vectorized = vectorizer.transform(X_preprocess)

    p = model.predict_classes(X_vectorized)[0][0]
    
    if p == 1:
        clas = 'pos'
    else:
        clas = 'neg'
    
    return [sentences_new, clas]

In [ ]:
experimental_data_text_purturbed = []
experimental_data_y_text_purturbed = []

for case_no_ in tqdm(random_cases):
    pt, c = text_purt_clas(case_no_)
    experimental_data_text_purturbed.append(pt)
    experimental_data_y_text_purturbed.append(c)

<h1>Experimental Data</h1>

In [ ]:
# for experimental_data_04_02_2021.xlsx 20 cases were used
experimental_data = pd.DataFrame()
experimental_data['idx'] = experimental_data_idx
experimental_data['text'] = experimental_data_text
experimental_data['y'] = experimental_data_y
experimental_data['y_hat'] = experimental_data_y_hat
experimental_data['text_purturbed'] = experimental_data_text_purturbed
experimental_data['y_text_purturbed'] = experimental_data_y_text_purturbed

In [ ]:
experimental_data.head()

In [ ]:
experimental_data.to_csv('experimental_data_04Feb2021.csv', index=False)